### 2. Clean data

Load and clean the .csv file using pandas. First, create a DataFrame `df` and check the number of rows. Then, remove duplicates by creating a new DataFrame `df_unique` and check the number of rows again.

In the example below, I use the file `elektriciens-vlaanderen.csv`, check the length of the data (rows) and apply the following changes.

* Remove duplicates
* Remove rows where Website is n/a
* Format string in the "Name" column to title case
* format the "Website" column: remove protocol (http/https), subdomain(www.) and "/" at end of URL, remove rows containing "facebook", remove rows containing shortened URL's (bitly)

Save df to .csv. I chose to overwrite the original to avoid creating too many files.

In [2]:
import pandas as pd

In [24]:
# Load the csv into a Pandas Dataframe
df = pd.read_csv("1-gathered-data/elektriciens-vlaanderen.csv", delimiter =",")
df.tail()

,Name,Website,Phone Number,City
35,"HELEC, HELECTRIEK BV",http://www.helec.be/,0474 71 29 78,Brugge
36,Ruben Janssens algemene elektriciteit,https://www.rj-elektriciteit.be/,0486 18 98 18,Brugge
37,Inlight,http://www.inlight-brugge.be/,0498 45 94 10,Brugge
38,siamand - algemene elektriciteitswerken,https://www.facebook.com/algemeneelektriciteit...,0465 15 67 34,Brugge
39,Mildan,NaN,0491 73 69 51,Brugge


In [25]:
df.shape[0]

40

In [27]:
# Remove duplicates
df = df.drop_duplicates(subset=['Name'])
df.shape[0]

40

In [26]:
df.isna().sum() #Checking how many empty cells are in the df

Name            0
Website         7
Phone Number    0
City            0
dtype: int64

In [28]:
# Remove rows not containing a URL, as this is crucial
df = df.dropna(subset=["Website"])
df.shape[0]

33

In [29]:
# Apply title string to "Name" column
df["Name"] = df["Name"].str.title()
df.head()

,Name,Website,Phone Number,City
0,Mm Elektro - Multitechnieken,http://www.mmelektro.be/,0486 40 39 08,Kortrijk
2,Electro Blondeel,http://www.electroblondeel.be/,0475 62 50 23,Kortrijk
3,Installatietechnieken Laverge,http://www.laverge.be/,0476 30 23 92,Kortrijk
4,Electro Devosco Bv,http://www.electrodevosco.be/,051 30 47 85,Kortrijk
6,Elektriciteitswerken Verhelst,http://www.elektriciteitverhelst.be/,0468 48 68 41,Kortrijk


In [32]:
# Remove rows containing "bit.ly" or "facebook"
df = df[~df['Website'].str.contains('facebook|bit.ly')]

# Remove protocol (http/https), subdomain (www.) and "/" at the end of URL
df['Website'] = df['Website'].str.replace(r'^https?://(www\.)?', '', regex=True).str.rstrip('/')

df.shape[0]

31

In [34]:
df.to_csv("1-gathered-data/elektriciens-vlaanderen.csv", index=None)

In [54]:
df.head()

,Name,Website,Phone Number,City,Email
0,Mm Elektro - Multitechnieken,mmelektro.be,0486 40 39 08,Kortrijk,info@mmelektro.be
1,Electro Blondeel,electroblondeel.be,0475 62 50 23,Kortrijk,None
2,Installatietechnieken Laverge,laverge.be,0476 30 23 92,Kortrijk,info@laverge.be
3,Electro Devosco Bv,electrodevosco.be,051 30 47 85,Kortrijk,None
4,Elektriciteitswerken Verhelst,elektriciteitverhelst.be,0468 48 68 41,Kortrijk,None


### 3. Search e-mail addresses

* Import the cleaned .csv file and scrape email addresses using BeautifulSoup and pandas.
*If your dataset is much larger, consider batching the data for more efficient processing.*

* Create a new Dataframe, containing the rows where scraping was succesful



In [109]:
import requests
from bs4 import BeautifulSoup

!pip install requests beautifulsoup4


def extract_email(website):
    # Voeg https://www. toe als de URL geen protocol bevat
    if not website.startswith("http"):
        website = "https://www." + website
    
    try:
        response = requests.get(website, timeout=5)
        response.raise_for_status()  # Raise an exception for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')
        email = None
        for a in soup.find_all('a', href=True):
            if "mailto:" in a['href']:
                email = a['href'].split("mailto:")[1].split("?")[0]
                break  # Stop searching after the first email address
        return email
    except requests.exceptions.RequestException as e:
        print(f"Error accessing website {website}: {e}")
        return None



# Load the CSV file into a pandas DataFrame
df = pd.read_csv("gathered-data/elektriciens-vlaanderen.csv") #this is the appended information, all provinces combined (Belgium)

# Create a new column to store extracted emails
df['Email'] = df['Website'].apply(extract_email)

# Save the updated DataFrame to a new CSV file
df.to_csv("gathered-data/elektriciens-vlaanderen-emails-scraped.csv", index=False)

Error accessing website https://www.electroblondeel.be: 403 Client Error: Forbidden for url: https://www.electroblondeel.be/
Error accessing website https://www.elektriciteitverhelst.be: HTTPSConnectionPool(host='www.elektriciteitverhelst.be', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:997)')))
Error accessing website https://www.adelectrics.be: HTTPSConnectionPool(host='www.adelectrics.be', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:997)')))
Error accessing website https://www.ew-claeys.be: HTTPSConnectionPool(host='www.ew-claeys.be', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: 

In [110]:
#Create a new dataframe, containing the rows with scraped values
df_with_emails = df.Email.dropna()
# Check how many rows the new df contains
df_with_emails.shape[0]

15

In [111]:
Percentage_scrape_succesful = (len(df_with_emails)/len(df)) * 100
print(f"{Percentage_scrape_succesful:.0f} % of scraping jobs were succesful.")

48 % of scraping jobs were succesful.


#### *Optional: Fill e-mail addresses

A large portion of scraping jobs will not be succesful. In this case we can make an educated guess by filling in the e-mail prefix before the website address.

* Use `info@` to ensure you use the general company e-mail.
* Avoid filling in `firstname.lastname@` or any variation as it is not allowed to send e-mails to personal mailboxes without consent.

*Or: Manually fill in the missing e-mail addresses in Excel by searching the web for best results.*

In [88]:
df_without_emails = df[df["Email"].isna()]
df_without_emails.shape[0]

16

In [96]:
# Function to generate Email addresses, based on URL
def generate_email(website):
    if pd.isna(website):
        return None  # Return None for missing values
        
    # Add 'info@' 
    email = 'info@' + website
    return email

df_with_emails_filled = df_without_emails.copy()

df_with_emails_filled["Email"] = df_with_emails_filled["Website"].apply(generate_email)

In [107]:
df_with_emails_filled

,Name,Website,Phone Number,City,Email
1,Electro Blondeel,electroblondeel.be,0475 62 50 23,Kortrijk,info@electroblondeel.be
3,Electro Devosco Bv,electrodevosco.be,051 30 47 85,Kortrijk,info@electrodevosco.be
4,Elektriciteitswerken Verhelst,elektriciteitverhelst.be,0468 48 68 41,Kortrijk,info@elektriciteitverhelst.be
6,Elektro Steve-O,steveoelektro.be,0487 73 50 51,Kortrijk,info@steveoelektro.be
7,Dupont Electro,electrodupont.be,056 50 38 57,Kortrijk,info@electrodupont.be
9,Vaernewyck Bvba,vaernewyck.be,0480 64 58 10,Kortrijk,info@vaernewyck.be
10,Ad Electrics,adelectrics.be,0472 48 32 73,Kortrijk,info@adelectrics.be
11,Debaled Technics,debaled.be,0468 34 49 13,Kortrijk,info@debaled.be
12,Decroix / Geert Leon,geert-decroix.be,056 77 23 10,Kortrijk,info@geert-decroix.be
13,Elkin,elkin.be,0496 46 06 47,Kortrijk,info@elkin.be


In [108]:
df_with_emails_filled.to_csv("gathered-data/elektriciens-vlaanderen-emails-filled.csv", index = None)